# GARPOS Notebook 
### This notebook demonstrates how to run the [GARPOS](https://github.com/s-watanabe-jhod/garpos) software in the EarthScope GeoLab cloud environment.  
11-07-2025

### GeoLab Basics

GeoLab is the name of EarthScope's Jupyter Hub, which allows users to run code via Jupyter notebooks (or in a terminal) on EarthScope resources in AWS.  

The `sfg-geolab` image comes pre-loaded with the software needed to process GNSS-A data. Each user has their own home directory `/home/jovyan` that is mounted to 
the image and any files kept there are accessibile only to that user and remain during future sessions.  

### Data 
For the purposes of this GARPOS demo, we have pre-processed some `cascadia-gorda` data and stored the shot data in an S3 bucket accessible to all users of the GeoLab hub.

Raw data is available via authenticated https from [https://data.earthscope.org/archive/seafloor](https://data.earthscope.org/archive/seafloor).

### Metadata
In order to simplify our interactions with the various metadata formats, we are translating all metadata into `site` and `vessel` json. We store and load these metadata from the S3 archive [https://data.earthscope.org/archive/seafloor/metadata](https://data.earthscope.org/archive/seafloor/metadata)

The metadata file is stored at /home/joyvan/[network]/[site]/[metadata]/

### Results
Results are stored per GARPOS run under `/home/joyvan/[network]/[site]/[campaign]/[survey]/GARPOS/results

### Logs
Some log output is piped to the notebook.  Additionally those logs plus debug logs are written to `/home/joyvan/data/sfg/[network]/[station]/logs`. 

In [ ]:
# GeoLab environment settings
# DEFAULT_CONFIG = {
#     "WORKING_ENVIRONMENT": "GEOLAB",
#     "MAIN_DIRECTORY_GEOLAB": "/home/jovyan", or local "/Users/terry/data/project/SeafloorGeodesy/GEOLABDemo",
#     "S3_SYNC_BUCKET": "seafloor-public-bucket-bucket83908e77-gprctmuztrim",
# }

import os

from es_sfgtools.config.env_config import Environment
from es_sfgtools.workflows.workflow_handler import WorkflowHandler
from earthscope_sdk import AsyncEarthScopeClient
from earthscope_sdk.config.settings import SdkSettings

# This will read the environment variables set above
Environment.load_working_environment()
print("Working Environment: " + os.environ["WORKING_ENVIRONMENT"])
print("Main Directory Geolab: " + os.environ["MAIN_DIRECTORY_GEOLAB"])
print("S3 Sync Bucket: " +  os.environ["S3_SYNC_BUCKET"])

## Grab temporary AWS credentials for the s3-seafloor role

In [ ]:
# Create an EarthScope client pointed at the dev environment
es = AsyncEarthScopeClient(settings=SdkSettings(profile_name="dev"))

# Set AWS credentials for S3 access
creds = await es.user.get_aws_credentials(role="s3-seafloor")
os.environ['AWS_ACCESS_KEY_ID'] = creds.aws_access_key_id
os.environ['AWS_SECRET_ACCESS_KEY'] = creds.aws_secret_access_key
os.environ['AWS_SESSION_TOKEN']  = creds.aws_session_token

## Select the data you are interested in running through GARPOS
Currently we have pre-processed the 2025 campaign for stations **GCC1**, **NBR1**, **NCC1** and **NTH1** in network **cascadia-gorda** and made shot data available to this environment in a shared S3 bucket.

Cascadia Campaigns
|        | GCC1        | NBR1        | NCC1        | NTH1        |
|--------|-------------|-------------|-------------|-------------|
| **2022** | 2022_A_1065 |             | 2022_A_1065 |             |
| **2023** | 2023_A_1063 | 2023_A_1063 | 2023_A_1063 |             |
| **2024** | 2024_A_1126 | 2024_A_1126 | 2024_A_1126 |             |
| **2025** | 2025_A_1126 | 2025_A_1126 | 2025_A_1126 | 2025_A_1126 |


In [ ]:
NETWORK = "cascadia-gorda"
STATION = "GCC1"
CAMPAIGN = "2025_A_1126"

## Set up the workflow with your chosen site/campaign

In [ ]:
workflow = WorkflowHandler()

workflow.set_network_station_campaign(
    network_id=NETWORK,
    station_id=STATION,
    campaign_id=CAMPAIGN
)

## Prepare data for GARPOS Processing 

Optional Variables to include in `mid_process_prep_garpos`
  * `site_metadata`: Site metadata or path to metadata file. If not provided, it will be loaded if available.
    * Default: None
  * `survey_id`: Survey identifier to process. If None, processes all surveys
    * Default: None
  * `custom_filters`: Custom filter settings for shot data preparation
    * Default: None
  * `override`: True/False. If true, re-prepares existing data.
    * Default: False
  * `write_intermetediate`: True/False, If true, writes intermediate files.
    * Default: False

A value error will be raised if site metadata is not loaded. 

### Optional: Set up custom filters

In [ ]:
# OPTIONAL: Edit the filter config as needed
FILTER_CONFIG = {
    "pride_residuals": {
        "enabled": False,
        "max_residual_mm": 8,
        "description": "Filter based on GNSS positioning residuals",
    },
    "max_distance_from_center": {
        "enabled": False,
        "max_distance_m": 150.0,
        "description": "Filter shots beyond maximum distance from array center",
    },
    "ping_replies": {
        "enabled": False,
        "min_replies": 1,
        "description": "Filter based on minimum acoustic ping replies",
    },
    "acoustic_filters": {
        "enabled": False,
        "level": "OK",
        "description": "Apply standard acoustic data quality filters",
    },
}

In [ ]:
# Prepare GARPOS data
workflow.midprocess_prep_garpos() 
# custom_filters=FILTER_CONFIG


## Run Garpos
This will loop through all the surveys in the campaign and run GARPOS on each.  

You can change the above inversion parameters and run again to compare results, but make sure to increment run_id so that the results are stored in a new directory.


Optional Variables to include in `modeling_run_garpos()`
  * `survey_id`: Optional survey identifier to process. If None, processes all surveys
    * Default: None
  * `run_id`: Identifier for the GARPOS run 
    * Default: "Test"
  * `iterations`: Number of GARPOS iterations to perform. 
    * Default: 1  
  * `site_metadata`: Site metadata or path to metadata file. If not provided, it will be loaded if available.
    * Default: None
  * `override`: True/False. If true, re-runs GARPOS even if results exist.
    * Default False
  * `custom_settings`: Custom GARPOS settings to apply
    * GARPOS settings
        * Inversion Parameters - for more information: https://github.com/s-watanabe-jhod/garpos/blob/master/sample/Settings-prep.ini
        * `spline_degree`, `log_lambda`, `log_gradlambda`, `mu_t`, `mu_mt`, `knotint0`, `knotint1`, `knotint2`, `rejectcriteria`, `inversiontype`, `positionalOffset`, `traveltimescale`, `maxloop`, `convcriteria`, `deltap`, `deltab`, `delta_center_position`

### Set up any custom settings


In [ ]:
# Example custom settings for GARPOS inversion
GARPOS_CUSTOM_SETTINGS = {
    'inversion_params' : {
        'maxloop': 5 # Maximum number of inversion loops
        }
    }

### Set up run ID and survey (if needed)

In [ ]:
RUN_ID = "1"   # Increment for each GARPOS run
SURVEY_NUMBER = 1
SURVEY_ID=f"{CAMPAIGN}_{SURVEY_NUMBER}"

### Run garpos with options outlined above

In [ ]:
workflow.modeling_run_garpos(run_id=RUN_ID, 
                             survey_id=SURVEY_ID, 
                             custom_settings=GARPOS_CUSTOM_SETTINGS)

## Plot Results

workflow.modeling_plot_garpos_results()